import libraries

In [4]:
from h3 import h3
import pandas as pd
import geopandas as gpd
import numpy as np
import time
import folium
from multiprocessing import Pool
from multiprocessing import cpu_count
import sys
nCORES = cpu_count()

import project functions

In [5]:
sys.path.append("..")
from src.geoIndexFunctions import *

### Get GBR boundary from GADM

In [7]:
pwd

'/home/lefteris/Desktop/trajectories/GeoIndex_POI_Trajectories/1 GeoIndexes'

In [10]:
SHP_PATH = '../data input/GADM/gadm36_GBR_shp/gadm36_GBR_2.shp'
gdf = gpd.read_file(SHP_PATH)

In [5]:
# pick the country you want to get the hexagons
COUNTRY = 'GBR'
# pick aperture size
APERTURE_SIZE = 7
# get only the columns of the unique id and geometry
COL_ID = 'GID_2'
COL_NAME = 'NAME_2'
COL_GEOM = 'geometry'
# number of cores to use
CORES = int(nCORES/1)

In [6]:
# get the country you want
gdf1 = gdf[gdf.GID_0==COUNTRY]
gdf1 = gdf1[[COL_ID, COL_GEOM, COL_NAME]]

In [7]:
print(len(gdf1))
gdf1.head()

183


,GID_2,geometry,NAME_2
0,GBR.1.1_1,"POLYGON ((-1.78996992 53.47292709, -1.79305696...",Barnsley
1,GBR.1.2_1,"POLYGON ((-2.68629479 51.31516266, -2.68065643...",Bath and North East Somerset
2,GBR.1.3_1,"POLYGON ((-0.58507907 52.11363983, -0.58805799...",Bedfordshire
3,GBR.1.4_1,"POLYGON ((-1.73416984 52.51014328, -1.73984325...",Birmingham
4,GBR.1.5_1,"POLYGON ((-2.51111197 53.63376617, -2.5131681 ...",Blackburn with Darwen


In [8]:
gdf1.plot()

In [9]:
# split the dataframe into chunks for parallelization
gdf1_split = np.array_split(gdf1, CORES)

In [10]:
from itertools import repeat

a_args = gdf1_split
second_arg = COL_GEOM
third_arg = COL_ID
fourth_arg = APERTURE_SIZE

zipped = zip(a_args, repeat(second_arg),repeat(third_arg),repeat(fourth_arg))

# pool.starmap for multiple arguments
with Pool(processes=CORES) as pool1:
    start = time.time()
    multi = pool1.starmap(get_hexagons_from_gdf, zipped)
    new_gdf1 = pd.concat(multi).reset_index(drop=True)
    end = round(time.time() - start, 3)

In [11]:
# pool.map for when the function is in the same script and arguments are global
# with Pool(processes=CORES) as pool1:
#     start = time.time()
#     new_gdf1 = pd.concat(pool1.map(get_hexagons_from_gdf, gdf1_split)).reset_index(drop=True)
#     end = round(time.time() - start, 3)

In [14]:
new_gdf1.head()

,GID_2,hex_list,hex_list_length
0,GBR.1.10_1,"[871942526ffffff, 871942500ffffff, 871942529ff...",78
1,GBR.1.11_1,"[87194a089ffffff, 87194a735ffffff, 87194a0d4ff...",16
2,GBR.1.12_1,"[871958391ffffff, 87195839cffffff, 871958769ff...",23
3,GBR.1.13_1,"[87195d020ffffff, 87195d126ffffff, 87195d11eff...",332
4,GBR.1.14_1,"[871951b26ffffff, 871951b23ffffff, 871951b22ff...",23


In [15]:
new_gdf1_geom = pd.merge(new_gdf1, gdf1, on='GID_2')

In [16]:
new_gdf1_geom.head()

,GID_2,hex_list,hex_list_length,geometry,NAME_2
0,GBR.1.10_1,"[871942526ffffff, 871942500ffffff, 871942529ff...",78,"POLYGON ((-1.72134185 53.90246201, -1.72350097...",Bradford
1,GBR.1.11_1,"[87194a089ffffff, 87194a735ffffff, 87194a0d4ff...",16,"POLYGON ((-0.12858801 50.88260269, -0.09664399...",Brighton and Hove
2,GBR.1.12_1,"[871958391ffffff, 87195839cffffff, 871958769ff...",23,"POLYGON ((-2.58329916 51.39482117, -2.6181612 ...",Bristol
3,GBR.1.13_1,"[87195d020ffffff, 87195d126ffffff, 87195d11eff...",332,"POLYGON ((-0.51059043 51.60243225, -0.49340501...",Buckinghamshire
4,GBR.1.14_1,"[871951b26ffffff, 871951b23ffffff, 871951b22ff...",23,"POLYGON ((-2.40541697 53.63182068, -2.3759954 ...",Bury


### flatten the hex_list 

In [17]:
#https://mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas-9849d8ff2401

In [18]:
flatten_new_gdf1 = new_gdf1_geom.hex_list.apply(pd.Series) \
.merge(new_gdf1_geom, left_index = True, right_index = True) \
.drop(["hex_list", 'hex_list_length'], axis = 1) \
.melt(id_vars = ['GID_2', 'geometry','NAME_2'], value_name = "hex") \
.drop("variable", axis = 1) \
.dropna()

In [19]:
flatten_new_gdf1[flatten_new_gdf1.GID_2=='GBR.1.10_1'].head()

,GID_2,geometry,NAME_2,hex
0,GBR.1.10_1,"POLYGON ((-1.72134185 53.90246201, -1.72350097...",Bradford,871942526ffffff
183,GBR.1.10_1,"POLYGON ((-1.72134185 53.90246201, -1.72350097...",Bradford,871942500ffffff
366,GBR.1.10_1,"POLYGON ((-1.72134185 53.90246201, -1.72350097...",Bradford,871942529ffffff
549,GBR.1.10_1,"POLYGON ((-1.72134185 53.90246201, -1.72350097...",Bradford,871942cd6ffffff
732,GBR.1.10_1,"POLYGON ((-1.72134185 53.90246201, -1.72350097...",Bradford,871942531ffffff


In [20]:
flatten_new_gdf1[flatten_new_gdf1.NAME_2.str.contains('Manchester')].head()

,GID_2,geometry,NAME_2,hex
53,GBR.1.54_1,"POLYGON ((-2.19558954 53.45247269, -2.18843269...",Manchester,871951b71ffffff
236,GBR.1.54_1,"POLYGON ((-2.19558954 53.45247269, -2.18843269...",Manchester,871951b29ffffff
419,GBR.1.54_1,"POLYGON ((-2.19558954 53.45247269, -2.18843269...",Manchester,871951b45ffffff
602,GBR.1.54_1,"POLYGON ((-2.19558954 53.45247269, -2.18843269...",Manchester,871951b59ffffff
785,GBR.1.54_1,"POLYGON ((-2.19558954 53.45247269, -2.18843269...",Manchester,871951a66ffffff


### Visualise both hexagons and actual boundaries 

In [21]:
visualise_hex_of_boundaryId_from_flatten_df('GBR.1.10_1','GBR.1.54_1',
                                    flatten_gdf=flatten_new_gdf1)
# need to fix to show all polylines

In [22]:
visualise_hex_of_boundaryId_from_flatten_df('GBR.1.10_1','GBR.1.54_1',
                                    flatten_gdf=flatten_new_gdf1, compaction = True)

# Next Steps

-Run the code for all boundaries within one country to get one aperture size hexagons
-Some of them will cross each other so these specific ones will be flagged as mutual hexagons and the points falling within those need to be indexed with the next finer aperture size
-At the end that process should run many times until there is no point falling in a mutual hexagon

### Appendix 

In [21]:
# def flaten(t,level=0):
#     l = []
#     for t1 in t:
#         if type(t1) is tuple:
#             if level == 0:
#                 l.append(flaten(t1,level+1))
#             else:
#                 l.extend(flaten(t1,level+1))
#         else:
#                 l.append(t1)
#     return l

# def get_hexagons_fromJson(boundary_geoJson, aperture=9):
#     # https://github.com/uber/h3-py/blob/master/h3/h3.py
#     hexagons = list(h3.polyfill(boundary_geoJson, aperture,geo_json_conformant=True))
#     return hexagons

# def get_hexagons_from_gdf(gdf):
#     # convert the multipolygons to polygons by our custom explode function
#     polygonOnly_gdf1 = good_explode(gdf)
#     polygonOnly_gdf1['hex_list_before'] = polygonOnly_gdf1[COL_GEOM].apply(lambda x : get_hexagons_fromJson(x.__geo_interface__))
#     polygonOnly_gdf1_group = polygonOnly_gdf1.groupby(COL_ID)
#     # count the sum of hexagons in each unique id
#     final_df = pd.DataFrame(polygonOnly_gdf1_group.apply(lambda grp: grp['hex_list_before'].sum()))
#     # reset the index and rename the 0 column
#     final_df.reset_index(inplace=True)
#     final_df.rename(columns={0:'hex_list'},inplace=True)
#     # count again the final number of hexagons for each unique id
#     final_df['hex_list_length'] = final_df['hex_list'].apply(lambda x : len(x))
#     return final_df

# def good_explode(df):
#         """
#         Explode muti-part geometries into multiple single geometries.
#         Each row containing a multi-part geometry will be split into
#         multiple rows with single geometries, thereby increasing the vertical
#         size of the GeoDataFrame.
#         The index of the input geodataframe is no longer unique and is
#         replaced with a multi-index (original index with additional level
#         indicating the multiple geometries: a new zero-based index for each
#         single part geometry per multi-part geometry).
#         Returns
#         -------
#         GeoDataFrame
#             Exploded geodataframe with each single geometry
#             as a separate entry in the geodataframe.
#         """
#         df_copy = df.copy()

#         exploded_geom = df_copy.geometry.explode().reset_index(level=-1)
# #         exploded_index = exploded_geom.columns[0]

#         df = pd.concat(
#             [df_copy.drop(df_copy._geometry_column_name, axis=1),
#              exploded_geom], axis=1)
#         # reset to MultiIndex, otherwise df index is only first level of
#         # exploded GeoSeries index.
# #         df.set_index(exploded_index, append=True, inplace=True)
# #         df.index.names = list(self.index.names) + [None]
#         geo_df = df.set_geometry(df._geometry_column_name)
        
#         return geo_df.drop(columns=['level_1']).reset_index(drop=True)

# def visualize_hexagon_polyline(hexagons, boundary_geoJson, folium_map=None):
#     #hexagons
#     polylines = []
#     lat = []
#     lng = []
#     for hex in hexagons:
#         polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
#         # flatten polygons into loops.
#         outlines = [loop for polygon in polygons for loop in polygon]
#         polyline = [outline + [outline[0]] for outline in outlines][0]
#         lat.extend(map(lambda v:v[0],polyline))
#         lng.extend(map(lambda v:v[1],polyline))
#         polylines.append(polyline)
        
#     # boundary  
#     first_boundary = boundary_geoJson['coordinates'][0]
#     if type(first_boundary)==tuple:
#         # convert tuple to list
#         boundary = flaten(first_boundary)
#     boundary.append(boundary[0])
#     final_boundary = [list(reversed(i)) for i in boundary]
#     lat = [p[0] for p in final_boundary]
#     lng = [p[1] for p in final_boundary]
    
#     if folium_map is None:
#         m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=10, tiles='cartodbpositron')
#     else:
#         m = folium_map
        
#     for polyline in polylines:
#         my_PolyLine=folium.PolyLine(locations=polyline,weight=4,opacity=0.6 ,color='red')
#         m.add_child(my_PolyLine)
        
#     my_Boundary=folium.PolyLine(locations=final_boundary,weight=4,opacity=0.6, color='green')
#     m.add_child(my_Boundary)
    
#     return m